# Something Something about Quantization

Faisal Qureshi    
faisal.qureshi@ontariotechu.ca

In [ ]:
from importlib import reload
import matplotlib.pyplot as plt
import torch

## Pre-trained model for MNIST idx 0

In [ ]:
import train

In [ ]:
reload(train)
image = train.pick_mnist_image(idx=0, root='../../data', download=False)
h, w = image.shape

plt.figure(figsize=(10,5))
plt.title(f'original image {w}x{h}')
plt.xticks([])
plt.yticks([])
plt.imshow(image, cmap='gray');

In [ ]:
import evaluate

In [ ]:
reload(evaluate)

model = evaluate.load_model('./nir.pth')
output_image = evaluate.reconstruct_image(model)
output_image -= output_image.min()
output_image /= output_image.max()

h, w = output_image.shape

plt.figure(figsize=(10,5))
plt.title(f'reconstructed {w}x{h}')
plt.xticks([])
plt.yticks([])
plt.imshow(output_image, cmap='gray');

## Image similarity

In [ ]:
import similarity

In [ ]:
reload(similarity)

i1 = torch.Tensor(image)
i1 = similarity.normalize(i1)
i2 = torch.Tensor(output_image)
i2 = similarity.normalize(i2)

print(f'mse  = {similarity.mse(i1, i2)}')
print(f'psnr = {similarity.psnr(i1, i2)}')

## Quantization in PyTorch

More at [https://pytorch.org/docs/stable/quantization.html](https://pytorch.org/docs/stable/quantization.html).

In [ ]:
model.to('cpu')
model.eval()
print(model)

In [ ]:
import os
import torch

def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("%.6f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

In [ ]:
model.qconfig = torch.quantization.get_default_qconfig('qnnpack')
torch.backends.quantized.engine = 'qnnpack'
model_tmp = torch.quantization.prepare(model, inplace=False)
modelq = torch.quantization.convert(model_tmp, inplace=False)

print_model_size(model)
print_model_size(modelq)

In [ ]:
output_image = evaluate.reconstruct_image(modelq, device='cpu')
output_image -= output_image.min()
output_image /= output_image.max()

h, w = output_image.shape

plt.figure(figsize=(10,5))
plt.title(f'reconstructed {w}x{h}')
plt.xticks([])
plt.yticks([])
plt.imshow(output_image, cmap='gray');

In [ ]:
reload(similarity)

i1 = torch.Tensor(image)
i1 = similarity.normalize(i1)
i2 = torch.Tensor(output_image)
i2 = similarity.normalize(i2)

print(f'mse  = {similarity.mse(i1, i2)}')
print(f'psnr = {similarity.psnr(i1, i2)}')

In [ ]:
torch.save(modelq.state_dict(),'1.pts')
torch.save(model.state_dict(),'2.pts')

In [ ]:
modelq.state_dict()